In [1]:
import numpy as np
from families import Gaussian, Bernoulli, Poisson
from glm import GLM

import statsmodels.api as sm
import statsmodels

/Users/matthewdrury/anaconda/envs/py36/lib/python3.6/site-packages/matplotlib/__init__.py:913: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))
/Users/matthewdrury/anaconda/envs/py36/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
N = 10000
X = np.empty(shape=(N, 3))
X[:, 0] = 1.0
X[:, 1] = np.random.uniform(size=N)
X[:, 2] = np.random.uniform(size=N)
nu = 1 - 2*X[:, 1] + X[:, 2]

## Linear Model

In [3]:
y = nu + np.random.normal(size=N)
model = GLM(family=Gaussian())
model.fit(X, y)

In [4]:
model.coef_

array([ 0.99792845, -1.96918849,  1.00254014])

In [5]:
model.parameter_covariance_

array([[  6.95279060e-04,  -6.01156046e-04,  -5.97529154e-04],
       [ -6.01156046e-04,   1.20692999e-03,   5.59830402e-06],
       [ -5.97529154e-04,   5.59830402e-06,   1.19674987e-03]])

In [6]:
np.sqrt(np.diag(model.parameter_covariance_))

array([ 0.02636814,  0.0347409 ,  0.03459407])

In [7]:
mod = sm.OLS(y, X)
res = mod.fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.289
Model:                            OLS   Adj. R-squared:                  0.289
Method:                 Least Squares   F-statistic:                     2034.
Date:                Sun, 27 Aug 2017   Prob (F-statistic):               0.00
Time:                        09:53:23   Log-Likelihood:                -14202.
No. Observations:               10000   AIC:                         2.841e+04
Df Residuals:                    9997   BIC:                         2.843e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.9979      0.026     37.846      0.0

## Linear Model with Sample Weights

In [8]:
sample_weights = np.random.uniform(0, 2, size=N)

In [9]:
model = GLM(family=Gaussian())
model = model.fit(X, y, sample_weights=sample_weights)

In [10]:
model.coef_

array([ 0.99631936, -1.94515315,  0.98255587])

## Logistic Model

In [11]:
p = 1 / (1 + np.exp(-nu))
y_logistic = np.random.binomial(1, p=p, size=N)

In [12]:
model = GLM(family=Bernoulli())
model.fit(X, y_logistic)

In [13]:
model.coef_

array([ 1.06465145, -2.15693403,  1.01416926])

In [14]:
model.dispersion_

2.0

In [15]:
model.parameter_covariance_

array([[ 0.00328087, -0.00302788, -0.00250253],
       [-0.00302788,  0.00608013, -0.00043436],
       [-0.00250253, -0.00043436,  0.00568654]])

In [16]:
np.sqrt(np.diag(model.parameter_covariance_) / model.dispersion_)

array([ 0.04050228,  0.05513679,  0.05332234])

In [17]:
mod = sm.Logit(y_logistic, X)
res = mod.fit()
print(res.summary())

Optimization terminated successfully.
         Current function value: 0.618298
         Iterations 5
                           Logit Regression Results                           
Dep. Variable:                      y   No. Observations:                10000
Model:                          Logit   Df Residuals:                     9997
Method:                           MLE   Df Model:                            2
Date:                Sun, 27 Aug 2017   Pseudo R-squ.:                 0.07513
Time:                        09:53:23   Log-Likelihood:                -6183.0
converged:                       True   LL-Null:                       -6685.2
                                        LLR p-value:                7.436e-219
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.0647      0.057     18.587      0.000       0.952       1.177
x1            -2.1569      0.

In [18]:
coefs = np.empty((1000, 3))
for i in range(1000):
    y_logistic = np.random.binomial(1, p=p, size=N)
    model = GLM(family=Bernoulli())
    model.fit(X, y_logistic)
    coefs[i, :] = model.coef_
    
print(coefs.std(axis=0))

[ 0.05829264  0.07839492  0.07587446]


## Poission Model

In [19]:
mu = np.exp(nu)
y_poisson = np.random.poisson(lam=mu, size=N)

In [20]:
model = GLM(family=Poisson())
model.fit(X, y_poisson)

In [21]:
model.coef_

array([ 1.02571278, -2.01328598,  0.9637053 ])

In [22]:
model.parameter_covariance_

array([[  3.38097091e-04,  -2.48183912e-04,  -3.54276616e-04],
       [ -2.48183912e-04,   7.24685856e-04,   4.14887593e-06],
       [ -3.54276616e-04,   4.14887593e-06,   6.10751177e-04]])

In [23]:
np.sqrt(np.diag(model.parameter_covariance_))

array([ 0.01838742,  0.02691999,  0.02471338])

In [24]:
mod = statsmodels.discrete.discrete_model.Poisson(y_poisson, X)
res = mod.fit()
print(res.summary())

Optimization terminated successfully.
         Current function value: 1.585818
         Iterations 7
                          Poisson Regression Results                          
Dep. Variable:                      y   No. Observations:                10000
Model:                        Poisson   Df Residuals:                     9997
Method:                           MLE   Df Model:                            2
Date:                Sun, 27 Aug 2017   Pseudo R-squ.:                  0.1966
Time:                        09:53:28   Log-Likelihood:                -15858.
converged:                       True   LL-Null:                       -19740.
                                        LLR p-value:                     0.000
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.0257      0.018     55.783      0.000       0.990       1.062
x1            -2.0133      0.

## Poisson with Exposures

In [25]:
mu = np.exp(nu)
expos = np.random.uniform(0, 10, size=N)
y_poisson = np.random.poisson(lam=(mu*expos), size=N)

In [26]:
model = GLM(family=Poisson())
model.fit(X, y_poisson, offset=np.log(expos))

In [27]:
model.coef_

array([ 1.0076394 , -2.01591812,  0.98564629])

## Linear Model with Correlated Predictors

In [28]:
N = 1000
X = np.empty(shape=(N, 3))
X[:, 0] = 1.0
X[:, 1] = np.random.uniform(size=N)
X[:, 2] = 0.9*X[:, 1] + np.random.uniform(-0.1, 0.1, size=N)
nu = 1 - 2*X[:, 1] + X[:, 2]

In [29]:
y = nu + np.random.normal(size=N)
model = GLM(family=Gaussian())
model.fit(X, y)

In [30]:
model.coef_

array([ 0.92748789, -1.30568126,  0.47360923])

In [31]:
model.parameter_covariance_

array([[  4.17523382e-03,  -5.98483390e-03,  -3.02194462e-04],
       [ -5.98483390e-03,   2.69732385e-01,  -2.87057070e-01],
       [ -3.02194462e-04,  -2.87057070e-01,   3.20282963e-01]])